In [37]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
!python3 -m spacy download en


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/preetgandhi95/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

    Linking successful
    /Users/preetgandhi95/miniconda3/envs/nlpclass/lib/python3.6/site-packages/en_core_web_sm
    -->
    /Users/preetgandhi95/miniconda3/envs/nlpclass/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [38]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [39]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [40]:
lyr=pd.read_csv('merged_eng_lyrics_genre_explicit.csv')

In [41]:
def repl(e):
    return [re.sub('\n', ' ', sent) for sent in e]

In [42]:
lyr=repl(lyr['lyrics'])

In [24]:
# blues=repl(list(lyr[lyr['Blues']==1]['lyrics']))
# cg=repl(list(lyr[lyr['Christian/Gospel']==1]['lyrics']))
# country=repl(list(lyr[lyr['Country']==1]['lyrics']))
# de=repl(list(lyr[lyr['Dance/Electro']==1]['lyrics']))
# disco=repl(list(lyr[lyr['Disco']==1]['lyrics']))
# dh=repl(list(lyr[lyr['Dutch-House']==1]['lyrics']))
# folk=repl(list(lyr[lyr['Folk']==1]['lyrics']))
# hiphop=repl(list(lyr[lyr['Hip-Hop']==1]['lyrics']))
# indie=repl(list(lyr[lyr['Indie']==1]['lyrics']))
# jazz=repl(list(lyr[lyr['Jazz']==1]['lyrics'] ) )
# latin=repl(list(lyr[lyr['Latin']==1]['lyrics']))
# metal=repl(list(lyr[lyr['Metal']==1]['lyrics']))
# other=repl(list(lyr[lyr['Other']==1]['lyrics']))
# pop=repl(list(lyr[lyr['Pop']==1]['lyrics']))
# rb=repl(list(lyr[lyr['R&B']==1]['lyrics']))
# reggae=repl(list(lyr[lyr['Reggae']==1]['lyrics']))
# ra=repl(list(lyr[lyr['Rock/Alt']==1]['lyrics']))
# rap=repl(list(lyr[lyr['Rock/Alt/Pop']==1]['lyrics']))
# missing=repl(list(lyr[lyr['missing']==1]['lyrics']))
# explicit=repl(list(lyr[lyr['Explicit']==1]['lyrics']))



In [43]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
lyr_words = list(sent_to_words(lyr))
# blues_words = list(sent_to_words(blues))
# cg_words = list(sent_to_words(cg))
# country_words = list(sent_to_words(country))
# de_words = list(sent_to_words(de))
# disco_words = list(sent_to_words(disco))
# dh_words = list(sent_to_words(dh))
# folk_words = list(sent_to_words(folk))
# hiphop_words = list(sent_to_words(hiphop))
# jazz_words = list(sent_to_words(jazz))
# latin_words = list(sent_to_words(latin))
# metal_words = list(sent_to_words(metal))
# other_words = list(sent_to_words(other))
# pop_words = list(sent_to_words(pop))
# rb_words = list(sent_to_words(rb))
# reggae_words = list(sent_to_words(reggae))
# ra_words = list(sent_to_words(ra))
# rap_words = list(sent_to_words(rap))
# missing_words = list(sent_to_words(missing))
# explicit_words = list(sent_to_words(explicit))

In [44]:
def build(wrd):
# Build the bigram and trigram models
    bigram = gensim.models.Phrases(wrd, min_count=1, threshold=1) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[wrd], threshold=1)  

# Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod,trigram_mod
    

In [45]:
lyr_b,lyr_t=build(lyr_words)
# blues_b,blues_t=build(blues_words)
# cg_b,cg_t=build(cg_words)
# country_b,country_t=build(country_words)
# de_b,de_t=build(de_words)
# disco_b,disco_t=build(disco_words)
# dh_b,dh_t=build(dh_words)
# folk_b,folk_t=build(folk_words)
# hiphop_b,hiphop_t=build(hiphop_words)
# jazz_b,jazz_t=build(jazz_words)
# latin_b,latin_t=build(latin_words)
# metal_b,metal_t=build(metal_words)
# other_b,other_t=build(other_words)
# pop_b,pop_t=build(pop_words)
# rb_b,rb_t=build(rb_words)
# reggae_b,reggae_t=build(reggae_words)
# ra_b,ra_t=build(ra_words)
# rap_b,rap_t=build(rap_words)
# missing_b,missing_t=build(missing_words)
# explicit_b,explicit_t=build(explicit_words)

/Users/preetgandhi95/miniconda3/envs/nlpclass/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [56]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

def make_bigrams(texts):
    return [lyr_b[doc] for doc in texts]

def make_trigrams(texts):
    return [lyr_t[lyr_b[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [57]:
def lem(data_words):# Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    return lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



In [58]:
lyr_lem=lem(lyr_words)
# blues_lem=lem(blues_words)
# cg_lem=lem(cg_words)
# country_lem=lem(country_words)
# de_lem=lem(de_words)
# disco_lem=lem(disco_words)
# dh_lem=lem(dh_words)
# folk_lem=lem(folk_words)
# hiphop_lem=lem(hiphop_words)
# jazz_lem=lem(jazz_words)
# latin_lem=lem(latin_words)
# metal_lem=lem(metal_words)
# other_lem=lem(other_words)
# pop_lem=lem(pop_words)
# rb_lem=lem(rb_words)
# reggae_lem =lem(reggae_words)
# ra_lem=lem(ra_words)
# rap_lem=lem(rap_words)
# missing_lem=lem(missing_words)
# explicit_lem=lem(explicit_words)

ValueError: [E088] Text of length 1696341 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [ ]:
data_lemmatized=lyr_lem

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]



In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis